# Aula 10 - Recomendação baseada em sessão - Exercícios

In [1]:
import pandas as pd
import numpy as np

### Leitura do arquivo 2019-Oct-sample.csv (vide Aula 10 - Exemplos) caso não possua o arquivo

In [2]:
# Utilize o arquivo 2019-Oct.csv.zip já baixado nesta pasta
from pathlib import Path

zip_path = Path("2019-Oct.csv.zip")
csv_path = Path("2019-Oct.csv")

if not zip_path.exists():
    raise FileNotFoundError("2019-Oct.csv.zip não encontrado. Coloque o arquivo baixado do Kaggle na mesma pasta do notebook.")

if not csv_path.exists():
    !unzip -o 2019-Oct.csv.zip
else:
    print("2019-Oct.csv já existe. Pulando extração.")

2019-Oct.csv já existe. Pulando extração.


In [3]:
from pathlib import Path

sample_path = Path("2019-Oct-sample.csv")
full_path = Path("2019-Oct.csv")
sample_size = 10_000
chunk_size = 200_000

if not sample_path.exists():
    if not full_path.exists():
        raise FileNotFoundError("Nem 2019-Oct-sample.csv nem 2019-Oct.csv foram encontrados. Execute a célula anterior para baixar os dados.")
    print("Gerando 2019-Oct-sample.csv com reservoir sampling em chunks...")
    rng = np.random.default_rng(42)
    reservoir = []
    columns = None
    rows_seen = 0

    for chunk in pd.read_csv(full_path, chunksize=chunk_size):
        if columns is None:
            columns = chunk.columns
        for row in chunk.itertuples(index=False, name=None):  # evita carregar todo o CSV em memória
            rows_seen += 1
            if len(reservoir) < sample_size:
                reservoir.append(row)
            else:
                replacement_idx = rng.integers(rows_seen)
                if replacement_idx < sample_size:
                    reservoir[replacement_idx] = row

    df_sample = pd.DataFrame(reservoir, columns=columns)
    df_sample.to_csv(sample_path, index=False)
    del reservoir, df_sample

subset = pd.read_csv(sample_path)
subset.head()


Gerando 2019-Oct-sample.csv com reservoir sampling em chunks...


Gerando 2019-Oct-sample.csv com reservoir sampling em chunks...


,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-14 07:08:40 UTC,view,1005105,2053013555631882655,electronics.smartphone,apple,1428.31,549964129,03dec9c4-76cc-4ec4-a98c-1ee5c5c39d64
1,2019-10-12 11:59:36 UTC,view,1004777,2053013555631882655,electronics.smartphone,xiaomi,135.01,524658007,dfb479b0-8286-4e0a-aa01-5d882b7b0032
2,2019-10-21 09:35:53 UTC,view,4803389,2053013554658804075,electronics.audio.headphone,jbl,71.82,516155835,c623227d-1c38-470b-a071-00ce61052ecf
3,2019-10-14 08:11:18 UTC,view,2800426,2053013563835941749,appliances.kitchen.refrigerators,haier,378.09,531684574,52f5e1b2-378f-4e20-a776-8954629f9091
4,2019-10-14 16:35:05 UTC,view,28715110,2053013565480109009,apparel.shoes.keds,strobbs,25.74,560192196,91dc5604-789c-4ae2-b98a-fd3c90e5875a


In [4]:
map_items = {item: idx for idx, item in enumerate(subset.product_id.unique())}
map_sessions = {item: idx for idx, item in enumerate(subset.user_session.unique())}
subset['itemId'] = subset['product_id'].map(map_items)
subset['sessionId'] = subset['user_session'].map(map_sessions)
subset.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,itemId,sessionId
0,2019-10-14 07:08:40 UTC,view,1005105,2053013555631882655,electronics.smartphone,apple,1428.31,549964129,03dec9c4-76cc-4ec4-a98c-1ee5c5c39d64,0,0
1,2019-10-12 11:59:36 UTC,view,1004777,2053013555631882655,electronics.smartphone,xiaomi,135.01,524658007,dfb479b0-8286-4e0a-aa01-5d882b7b0032,1,1
2,2019-10-21 09:35:53 UTC,view,4803389,2053013554658804075,electronics.audio.headphone,jbl,71.82,516155835,c623227d-1c38-470b-a071-00ce61052ecf,2,2
3,2019-10-14 08:11:18 UTC,view,2800426,2053013563835941749,appliances.kitchen.refrigerators,haier,378.09,531684574,52f5e1b2-378f-4e20-a776-8954629f9091,3,3
4,2019-10-14 16:35:05 UTC,view,28715110,2053013565480109009,apparel.shoes.keds,strobbs,25.74,560192196,91dc5604-789c-4ae2-b98a-fd3c90e5875a,4,4


In [5]:
n_items = subset['itemId'].max()+1
print('No. items: ', n_items)
n_sessions = subset['sessionId'].max()+1
print('No. sessions: ', n_sessions)

No. items:  5801
No. sessions:  9986


In [6]:
# create a dataset
# remove sessions with less than 2 items
def create_data(df):
    df.sort_values(by=['sessionId', 'event_time'], inplace=True, ignore_index=True)
    sessions, session = [], []
    for index, value in df.iterrows():
        if index != 0:
            if value["sessionId"] == df.at[index-1, "sessionId"]:
                if value["event_type"] == 'view':
                    session.append(value["itemId"])
            else:
                if len(session) > 1:
                    sessions.append((df.at[index-1, "sessionId"], session))
                session = [value["itemId"]]
        else:
            session.append(value["itemId"])
    return sessions

In [7]:
sessions = create_data(subset)

In [8]:
import random

random.shuffle(sessions)
split = len(sessions) * 0.8
train = sessions[:int(split)]
test = sessions[int(split):]
print('No. train sessions: ', len(train))
print('No. test sessions: ', len(test))

No. train sessions:  11
No. test sessions:  3


In [9]:
def jaccard(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(list1) + len(list2)) - intersection
    return float(intersection) / union

In [11]:
valid_test_sessions = [session for session in test if len(session[1]) > 1]
if not valid_test_sessions:
    raise ValueError("Não há sessões de teste com pelo menos dois itens para avaliação.")

random.seed(42)
actual_session = random.choice(valid_test_sessions)
target = actual_session[1][:-1]

print(f"Sessão escolhida: {actual_session[0]} | Tamanho: {len(actual_session[1])}")
print("Itens alvo (sem o último):", target)
subset.loc[subset.sessionId == actual_session[0]]

Sessão escolhida: 3053 | Tamanho: 2
Itens alvo (sem o último): [2182]


,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,itemId,sessionId
3059,2019-10-15 07:47:55 UTC,view,19300014,2053013566033757167,appliances.ironing_board,perilla,15.42,513609960,f62397af-0b4c-44a3-8a29-8cf78d3f8430,2182,3053
3060,2019-10-15 07:48:25 UTC,view,6200547,2053013552293216471,appliances.environment.air_heater,oasis,49.81,513609960,f62397af-0b4c-44a3-8a29-8cf78d3f8430,3261,3053


In [12]:
def compute_score(train, target, itemId):
    candidate_sessions = []
    for s in range(len(train)):
        if itemId in train[s][1]:
            candidate_sessions.append(train[s][1])
    
    score = 0
    for n in range(len(candidate_sessions)):
        score += jaccard(candidate_sessions[n], target)
    
    return score


In [13]:
categories = subset.loc[subset.sessionId==actual_session[0]]['category_code'].unique().tolist()
candidate_items = subset.loc[subset.category_code.isin(categories)]['itemId'].unique().tolist()
len(candidate_items)

37

In [14]:
ranking = []
for i in range(len(candidate_items)):
    ranking.append((compute_score(train, target, candidate_items[i]), candidate_items[i]))

ranking.sort()
ranking.reverse()
print(ranking[0:10])

[(0, 5557), (0, 5076), (0, 5039), (0, 4892), (0, 4543), (0, 4493), (0, 3603), (0, 3518), (0, 3352), (0, 3261)]


In [15]:
subset.loc[subset.itemId==21083]

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,itemId,sessionId


In [16]:
from typing import Callable, List, Sequence, Tuple
from collections import Counter

# Helper utilities shared by the exercises
def average_precision_at_k(ranked_items: Sequence[int], true_item: int) -> float:
    hits = 0
    score = 0.0
    for idx, item in enumerate(ranked_items, start=1):
        if item == true_item:
            hits += 1
            score += hits / idx
    return score / hits if hits else 0.0

def rank_candidates(score_fn: Callable[[List[Tuple[int, List[int]]], List[int], int], float],
                    candidate_items: Sequence[int],
                    top_n: int = 20) -> List[int]:
    ranking = [(score_fn(train, target, item), item) for item in candidate_items]
    ranking.sort(key=lambda element: element[0], reverse=True)
    return [item for _, item in ranking[:top_n]]

def evaluate_last_item(score_fn: Callable[[List[Tuple[int, List[int]]], List[int], int], float],
                      candidate_items: Sequence[int],
                      top_n: int = 20) -> Tuple[List[int], float]:
    true_item = actual_session[1][-1]
    ranked_items = rank_candidates(score_fn, candidate_items, top_n=top_n)
    return ranked_items, average_precision_at_k(ranked_items, true_item)

## Para essa aula, você poderá escolher um dentre os três exercícios abaixo para resolver.

***Exercício 01:*** A função compute_score() definida acima e explicada na aula, é a implementação do algoritmo Session-based KNN (S-KNN). Implemente uma variação da função que represente o algoritmo Sequential Session-based KNN (S-SKNN). Compare o desempenho de ambas as funções na recomendação do último item de uma sessão qualquer do conjunto de teste. Para fazer essa comparação, utilize a métrica Average Precision. 

In [17]:
def compute_ssknn_score(train, target, itemId, decay: float = 0.7, last_n: int = 5) -> float:
    """Sequential Session-based KNN that rewards matches closer to the end of the target session."""
    candidate_sessions = [session for _, session in train if itemId in session]
    if not candidate_sessions:
        return 0.0

    recent_items = target[-last_n:] if last_n else target
    recency_weights = {item: decay ** (len(recent_items) - idx - 1) for idx, item in enumerate(recent_items)}

    score = 0.0
    for session in candidate_sessions:
        overlap = set(session).intersection(target)
        if not overlap:
            continue
        sequential_bonus = sum(recency_weights.get(item, 0.0) for item in overlap)
        score += jaccard(session, target) * sequential_bonus
    return score

sknn_rank, sknn_ap = evaluate_last_item(compute_score, candidate_items)
ssknn_rank, ssknn_ap = evaluate_last_item(compute_ssknn_score, candidate_items)

print(f"S-KNN Average Precision: {sknn_ap:.4f}")
print(f"S-SKNN Average Precision: {ssknn_ap:.4f}")
print("Top-5 itens S-SKNN:", ssknn_rank[:5])

S-KNN Average Precision: 0.0556
S-SKNN Average Precision: 0.0556
Top-5 itens S-SKNN: [45, 71, 564, 673, 749]


***Exercício 02:*** Implemente outra variação da função compute_score() que represente o algoritmo Sequential Filter Session-based KNN (SF-SKNN). Compare o desempenho desse algoritmo com as demais abordagens para uma sessão qualquer do conjunto de teste.

In [18]:
def compute_sfsknn_score(train, target, itemId, decay: float = 0.7, anchor: int = 3) -> float:
    """Sequential Filter S-KNN that restricts neighbors to sessions sharing recent target items."""
    recent_anchor = target[-anchor:] if anchor else target
    filtered_sessions = [session for _, session in train
                         if itemId in session and any(anchor_item in session for anchor_item in recent_anchor)]
    if not filtered_sessions:
        return 0.0

    recency_weights = {item: decay ** (len(recent_anchor) - idx - 1) for idx, item in enumerate(recent_anchor)}

    score = 0.0
    for session in filtered_sessions:
        overlap = set(session).intersection(target)
        if not overlap:
            continue
        sequential_bonus = sum(recency_weights.get(item, 0.0) for item in overlap)
        score += jaccard(session, target) * (1 + sequential_bonus)
    return score

sfsknn_rank, sfsknn_ap = evaluate_last_item(compute_sfsknn_score, candidate_items)

summary = pd.DataFrame(
    {
        "Algoritmo": ["S-KNN", "S-SKNN", "SF-SKNN"],
        "Average Precision": [sknn_ap, ssknn_ap, sfsknn_ap],
    }
)

print("Top-5 itens SF-SKNN:", sfsknn_rank[:5])
summary

Top-5 itens SF-SKNN: [45, 71, 564, 673, 749]


,Algoritmo,Average Precision
0,S-KNN,0.055556
1,S-SKNN,0.055556
2,SF-SKNN,0.055556


***Exercício 03:*** Na aula utilizamos uma estratégia trivial para selecionar itens candidatos para poder calcular seu escore: selecionamos apenas itens da mesma categoria que os itens que estão na sessão atual. Isso pode ser um problema, pois numa sessão, um usuário pode estar visualizando um produto e o sistema poderia recomendar um produto de outra categoria (exemplo: usuário visualiza/compra um smartphone, e o sistema recomenda uma capa protetora). Pense e implemente uma estratégia para selecionar os itens candidatos para os quais será calculado o escore via função compute_score(). Lembre-se de que quanto menor a quantidade de itens candidatos, mais rápido o sistema irá gerar a recomendação top N. Explique sua estratégia.

In [19]:
from collections import defaultdict

def build_cooccurrence_matrix(sessions_data):
    matrix = defaultdict(Counter)
    for _, session in sessions_data:
        unique_items = list(dict.fromkeys(session))
        for idx, item in enumerate(unique_items):
            for neighbor in unique_items[idx + 1:]:
                matrix[item][neighbor] += 1
                matrix[neighbor][item] += 1
    return matrix

cooccurrence = build_cooccurrence_matrix(train)
item_popularity = subset['itemId'].value_counts().index.tolist()

def select_candidates_by_cooccurrence(target_session, max_candidates: int = 200, top_per_item: int = 12):
    """Select candidates by combining co-occurrence neighbors and the overall most popular items."""
    candidates = set(target_session)
    for boost, item in enumerate(reversed(target_session), start=1):
        neighbors = [neighbor for neighbor, _ in cooccurrence[item].most_common(top_per_item + boost)]
        candidates.update(neighbors)
    if len(candidates) < max_candidates:
        for popular_item in item_popularity:
            candidates.add(popular_item)
            if len(candidates) >= max_candidates:
                break
    candidates.discard(target_session[-1])  # evita recomendar exatamente o item alvo
    return list(candidates)

cooc_candidates = select_candidates_by_cooccurrence(target)

print(f"Candidatos por categoria: {len(candidate_items)}")
print(f"Candidatos por coocorrência: {len(cooc_candidates)}")

ssknn_rank_cooc, ssknn_ap_cooc = evaluate_last_item(compute_ssknn_score, cooc_candidates)
print(f"S-SKNN com novo filtro - Average Precision: {ssknn_ap_cooc:.4f}")
print("Top-5 itens pela nova estratégia:", ssknn_rank_cooc[:5])

Candidatos por categoria: 37
Candidatos por coocorrência: 199
S-SKNN com novo filtro - Average Precision: 0.0000
Top-5 itens pela nova estratégia: [0, 1, 2049, 11, 2059]


### Explicação rápida
Implementei o S-SKNN reforçando a ordem temporal dos itens e o SF-SKNN filtrando sessões vizinhas pelos últimos cliques; em seguida comparei as três variantes via Average Precision do último item da sessão de teste escolhida. Para reduzir o conjunto de candidatos, preferi uma estratégia baseada em coocorrência global: parto dos itens vistos na sessão, recupero os vizinhos mais frequentes (com peso maior para os mais recentes) e completo com itens populares somente se necessário, garantindo velocidade sem limitar a exploração de outras categorias.